In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [90]:
df= pd.read_csv('House_Rent_Dataset.csv')
df.head()

,BHK,Rent,Size,Floor,AreaType,AreaLocality,City,FurnishingStatus,TenantPreferred,Bathroom,PointofContact
0,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [106]:
df.corr()

<ipython-input-106-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,BHK,Rent,Size,Bathroom
BHK,1.000000,0.369718,0.716145,0.794885
Rent,0.369718,1.000000,0.413551,0.441215
Size,0.716145,0.413551,1.000000,0.740703
Bathroom,0.794885,0.441215,0.740703,1.000000


In [92]:
  X = df.drop("Rent",axis=1)
y= df["Rent"]
X.drop("AreaLocality",axis=1,inplace =True)

In [93]:
X.shape

(4746, 9)

Floor has Ground, Lower and Upper basements and other floors.
Lets do some feature engineering

In [94]:
X["Floor"] = X["Floor"].apply(lambda x: x.split()[0])
X["Floor"] = X["Floor"].replace({"Ground": 0, "Lower": -1, "Upper": 0.5})
X["Floor"] = X["Floor"].astype("int64")

In [95]:
Q1 = y.quantile(0.25)
Q3 = y.quantile(0.80)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

y = y.clip(lower_bound, upper_bound)

In [96]:
result = pd.concat([X, y], axis=1)

In [97]:
result.drop_duplicates(inplace=True)

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.33,random_state= 42)

In [99]:
cat_col = X.select_dtypes(include=['object']).columns
num_col = X.select_dtypes(exclude=['object']).columns

In [100]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [101]:
cat_pipe= Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("encoder",OneHotEncoder())
    ]
)

num_pipe= Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)

In [102]:
fe = ColumnTransformer([
    ("cat_pipe",cat_pipe, cat_col),
    ("num_pipe",num_pipe, num_col)
])

In [103]:
X_train = fe.fit_transform(X_train)
X_test = fe.transform(X_test)

In [104]:
X_train.shape,X_test.shape

((3179, 22), (1567, 22))

In [105]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

models = [
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor()
]

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"{type(model).__name__} R-squared: {r2:.4f}")


LinearRegression R-squared: 0.7948
DecisionTreeRegressor R-squared: 0.7205
RandomForestRegressor R-squared: 0.8502


In [194]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Create a Random Forest regressor object
rf_reg = RandomForestRegressor()

# Create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(rf_reg, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and R-squared score
print("Best hyperparameters: ", grid_search.best_params_)
print("R-squared score: ", grid_search.best_score_)


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors an

Best hyperparameters:  {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 500}
R-squared score:  0.8214980153459365


In [70]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

input_dim = X_train.shape[1]

model = Sequential()

model.add(Dense(256, input_dim=input_dim, activation='relu'))

model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='LeakyReLU'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.001))

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


80/80 [==============================] - 1s 5ms/step - loss: 24630.3066 - val_loss: 23320.6426
Epoch 2/100
80/80 [==============================] - 0s 3ms/step - loss: 18768.5488 - val_loss: 12503.6514
Epoch 3/100
80/80 [==============================] - 0s 4ms/step - loss: 10850.1553 - val_loss: 7869.2549
Epoch 4/100
80/80 [==============================] - 0s 4ms/step - loss: 7975.6807 - val_loss: 7022.2007
Epoch 5/100
80/80 [==============================] - 0s 3ms/step - loss: 7404.4009 - val_loss: 6668.3457
Epoch 6/100
80/80 [==============================] - 0s 4ms/step - loss: 7179.0898 - val_loss: 6471.8682
Epoch 7/100
80/80 [==============================] - 0s 4ms/step - loss: 6894.1650 - val_loss: 6389.2349
Epoch 8/100
80/80 [==============================] - 0s 4ms/step - loss: 6855.3457 - val_loss: 6397.0381
Epoch 9/100
80/80 [==============================] - 0s 4ms/step - loss: 6732.7573 - val_loss: 6248.6201
Epoch 10/100
80/80 [==============================] - 0s 4ms/s

In [71]:
y_pred= model.predict(X_test)

from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)



49/49 [==============================] - 0s 1ms/step
R-squared score: 0.8105835821388083
